In [ ]:
import numpy as np
import pandas as pd
import sobol_seq
from string import ascii_lowercase
import numba
import matplotlib.pyplot as plt

k = 6 # number of parameters inquired

# It is assumed the sm represents the dataframe on which we will be operating
a2 = [0, 0.5, 3, 9, 99, 99]
b3 = [6.52, 6.52, 6.52, 6.52, 6.52, 6.52]

@numba.jit    
def A1(sm):
    dummyA1 = pd.DataFrame()
    for j in range(0,k):
        dummyA1[j] = np.prod(sm[sm.columns[0:j+1]], axis = 1)*(-1)**(j+1)
    return dummyA1.sum(axis=1)
        
def A2(sm):
    dummyA2 = pd.DataFrame()
    for j in range(0,k):
        dummyA2[j] = (abs(4*sm[sm.columns[j]]-2)+a2[j])/(1+a2[j])
    return (dummyA2.product(axis=1))

functions = [A1, A2]

#AEA1

def Ek2(k):
    return((1/6)*(1-(1/3)**k)+(4/15)*((-1)**(k+1)*(1/2)**k+(1/3)**k))

def V(k):
    return((1/10)*(1/3)**k+(1/18)-(1/9)*(1/2)**(2*k)+(-1)**(k+1)*(2/45)*(1/2)**k)

for j in range(0, k):
    def Ej(j):
        return((1/6)*(1-(1/3)**(j))+(4/15)*((-1)**(j+1)*(1/2)**(j)+(1/3)**(j)))
              
    def T1(j):
        return((1/2)*((1/3)**(j-1)*(1-(1/3)**(k-j))))

    def T2(j):
        return((1/2)*((1/3)**j-(1/3)**k))
    
    def T3(j):
        return((3/5)*(4*(1/3)**(k+1)+(-1)**(j+k+1)*(1/2)**(k-j-2)*(1/3)**(j+1)))
    
    def T4(j):
        return((1/5)*((-1)**(j+2)*(1/3)*(1/2)**(j-2)-4*(1/3)**(j+1)))
    
    def T5(j):
        return((1/5)*((-1)**(k+1)*(1/3)*(1/2)**(k-2)+(-1)**(k+j)*(1/3)**(j+1)*(1/2)**(k-j-2)))
    
def A1ST(j):
    return((Ek2(k)-Ej(j)-(1/4)*(T1(j)-2*T2(j)+T3(j))-T4(j)-T5(j))/V(k))

#ÁEA2

def VjA2(j):
    return((1/3)/(1+a2[j])**2)

def VA2(j):
    productA2 = []
    for j in range(0, k):
        productA2.append(1+VjA2(j))
    return np.product(productA2)-1

def VnA2(j):
    return((VA2(j)+1)/(1+VjA2(j)))

def VTjA2(j):
    return VjA2(j)*(VnA2(j))

def A2ST(j):
    return(VTjA2(j)/VA2(j))

def create_dict(key, values):
    return dict(zip(key, values))

analyticalValues = [A1ST, A2ST]

AE = []
AE_names = []
for w in analyticalValues:
    for j in range (0,k):
        AE.append(w(j))
        AE_names.append('AE' + str(w.__name__) + str(j+1))
AE_dic = create_dict(AE_names, AE)

letters = [l for l in ascii_lowercase]

p = 14
run = 50

n = 2

In [ ]:
srun = [2**s*int(run*((p-s-1)+k*(s-2))/(p-3)) for s in range(2,p)]
srun.insert(0,0)
srunc = [2**s*int(run*(k*(s-2))/(p-3)) for s in range(2,p)]
srunr = [2**s*int(run*(p-s-1)/(p-3)) for s in range(2,p)]
df = pd.DataFrame(sobol_seq.i4_sobol_generate(k, sum(srun))).sample(frac=1).reset_index(drop=True)

In [ ]:
p_sample = []
p_sample_name = []

qamples = [df.iloc[srun[s-2]:(srun[s-2]+srun[s-1])].reset_index(drop=True) for s in range(2,p)]

run_samples = []
run_c_samples = []
for s in range (2, p-1):
    run_samples.append(qamples[s-2].iloc[:2**s*int(run*((p-s-1))/(p-3))].reset_index(drop=True))
    
    Check_mean = []
    run_samplesr = []
    for r in range (int(run*((p-s-1))/(p-3))):
        run_samplesr.append(run_samples[s-2].iloc[r*int(len(run_samples[s-2])/int(run*(p-s-1)/(p-3))):(r+1)*\
        int(len(run_samples[s-2])/int(run*(p-s-1)/(p-3)))].reset_index(drop=True))
        
        sample_Matrices = []
        for m in range (n):
            sample_Matrices.append(run_samplesr[r].iloc[int(m*(len(run_samplesr[r])/n)):int((m+1)*\
            (len(run_samplesr[r])/n))].reset_index(drop=True))
                
        sample_Matrices_dic = create_dict(letters, sample_Matrices)
        
        mixed_Matrices = []
        mm_names = []
        for sm in range (0,len(sample_Matrices)):
            for sm1 in range (0,len(sample_Matrices)):
                if sm == sm1:
                    continue
                else:
                    for c in sample_Matrices[sm]:
                        mixed_Matrices.append(sample_Matrices[sm].copy())
                        mixed_Matrices[len(mixed_Matrices)-1][c]=sample_Matrices[sm1].copy()[c]
                        mm_names.append(str(letters[sm] + letters[sm1] + str(c+1)))
        mixed_Matrices_dic = create_dict(mm_names, mixed_Matrices)
        
        matrices_dic = {**sample_Matrices_dic, **mixed_Matrices_dic}
        
        names = []
        names1 = []
        names2 = []
        values = []
        values1 = []
        values2 = []
        for f in functions:
            for ss, zs in sample_Matrices_dic.items():
                names.append(f.__name__+str(ss))
                values.append(f(zs))

            for sq, zq in mixed_Matrices_dic.items():
                names1.append(f.__name__+str(sq))
                values1.append(f(zq))
            
            for sM, zM in matrices_dic.items():
                names2.append(f.__name__+str(sM))
                values2.append(f(zM))
                
        f_SM_dic = create_dict(names, values)
        f_MM_dic = create_dict(names1, values1)
        f_matrices_dic = create_dict(names2, values2)

        Check=[]
        CheckName = []
        for f in functions:
            for j in range(1,k+1):
                difference = []
                for mk, mz in f_matrices_dic.items():
                    if mk[0:2]==f.__name__:
                        validkeys3 = []
                        for fk1 in f_MM_dic.keys():
                            if len(mk)==3: 
                                if mk[2] == 'a' and fk1[0:3]==mk[0:3] and fk1[-1]==str(j):
                                    validkeys3.append(fk1)
                            else: 
                                if fk1[0:3]==mk[0:3] and fk1[-1]==mk[-1] and fk1!=mk and fk1>=mk and fk1[-1]==str(j):
                                    validkeys3.append(fk1)
                        z1 = dict(filter(lambda i:i[0] in validkeys3, f_MM_dic.items()))
                        for zk, zv in z1.items():
                            difference.append(0.5*(((mz-zv)**2).mean())/mz.var())
                Check.append(sum(difference)/len(difference))
                CheckName.append('CheckMAE'+ str(f.__name__) + str(j))
        Check_dic = create_dict(CheckName, Check)
        Check_mean.append(Check_dic)
    CheckMAE_mean_dic = {Jmk1:[Check_mean[Jmv1][Jmk1] for Jmv1 in range(len(Check_mean))] for Jmk1 in Check_mean[0].keys()}
    for Jmk1,Jmv1 in CheckMAE_mean_dic.items():
        CheckMAE_mean_dic[Jmk1] = sum(CheckMAE_mean_dic[Jmk1])/len(CheckMAE_mean_dic[Jmk1])
    p_sample.append(CheckMAE_mean_dic)
    p_sample_name.append((k+1)*2**s)
p_sample_dic = create_dict(p_sample_name, p_sample)

In [ ]:
Checkq = pd.DataFrame(p_sample_dic)
ExtraA1=(((Checkq.iloc[:int(len(Checkq)/n)]/Checkq.iloc[:int(len(Checkq)/n)].sum()))).rename(columns=lambda x: x*2)
ExtraA2=(((Checkq.iloc[int(len(Checkq)/n):]/Checkq.iloc[int(len(Checkq)/n):].sum()))).rename(columns=lambda x: x*2)
ExtraA1=(ExtraA1*srunc[1:]).astype(int)
ExtraA2=(ExtraA2*srunc[1:]).astype(int)
ExtraA1.insert(loc=0, column=28, value=0)
ExtraA2.insert(loc=0, column=28, value=0)
ExtraA1 = ((ExtraA1+srunr)*(k+1)/ExtraA1.columns.values).astype(int)
ExtraA2 = ((ExtraA2+srunr)*(k+1)/ExtraA2.columns.values).astype(int)
Extra = ExtraA1.append(ExtraA2)

In [ ]:
p_sample_name = []
p_sample = []
run_samples = []
run_c_samples = []
for s in range (2, p):
    
    Check_mean = []
    run_samplesr = []
    
    for r in range (int(run*((p-s-1)+k*(s-2))/(p-3))):
        run_samplesr.append(qamples[s-2].iloc[r*int(len(qamples[s-2])/int(run*((p-s-1)+k*(s-2))/(p-3))):(r+1)*\
        int(len(qamples[s-2])/int(run*((p-s-1)+k*(s-2))/(p-3)))].reset_index(drop=True))
        
        sample_Matrices = []
        for m in range (n):
            sample_Matrices.append(run_samplesr[r].iloc[int(m*(len(run_samplesr[r])/n)):int((m+1)*\
            (len(run_samplesr[r])/n))].reset_index(drop=True))
                
        sample_Matrices_dic = create_dict(letters, sample_Matrices)
        
        mixed_Matrices = []
        mm_names = []
        for sm in range (0,len(sample_Matrices)):
            for sm1 in range (0,len(sample_Matrices)):
                if sm == sm1:
                    continue
                else:
                    for c in sample_Matrices[sm]:
                        mixed_Matrices.append(sample_Matrices[sm].copy())
                        mixed_Matrices[len(mixed_Matrices)-1][c]=sample_Matrices[sm1].copy()[c]
                        mm_names.append(str(letters[sm] + letters[sm1] + str(c+1)))
        mixed_Matrices_dic = create_dict(mm_names, mixed_Matrices)
        
        matrices_dic = {**sample_Matrices_dic, **mixed_Matrices_dic}
        
        names = []
        names1 = []
        names2 = []
        values = []
        values1 = []
        values2 = []
        for f in functions:
            for ss, zs in sample_Matrices_dic.items():
                names.append(f.__name__+str(ss))
                values.append(f(zs))

            for sq, zq in mixed_Matrices_dic.items():
                names1.append(f.__name__+str(sq))
                values1.append(f(zq))
            
            for sM, zM in matrices_dic.items():
                names2.append(f.__name__+str(sM))
                values2.append(f(zM))
                
        f_SM_dic = create_dict(names, values)
        f_MM_dic = create_dict(names1, values1)
        f_matrices_dic = create_dict(names2, values2)

        Check=[]
        CheckName = []
        for f in functions:
            for j in range(1,k+1):
                difference = []
                for mk, mz in f_matrices_dic.items():
                    if mk[0:2]==f.__name__:
                        validkeys3 = []
                        for fk1 in f_MM_dic.keys():
                            if len(mk)==3: 
                                if mk[2] == 'a' and fk1[0:3]==mk[0:3] and fk1[-1]==str(j):
                                    validkeys3.append(fk1)
                            else: 
                                if fk1[0:3]==mk[0:3] and fk1[-1]==mk[-1] and fk1!=mk and fk1>=mk and fk1[-1]==str(j):
                                    validkeys3.append(fk1)
                        z1 = dict(filter(lambda i:i[0] in validkeys3, f_MM_dic.items()))
                        for zk, zv in z1.items():
                            difference.append(0.5*(((mz-zv)**2).mean())/mz.var())
                Check.append(sum(difference)/len(difference))
                CheckName.append('CheckMAE'+ str(f.__name__) + str(j))
        Check_dic = create_dict(CheckName, Check)
        Check_mean.append(Check_dic)
    p_sample.append(Check_mean)
    p_sample_name.append((k+1)*2**(s-1))
p_sample_dic = create_dict(p_sample_name, p_sample)

In [ ]:
runsdf = [pd.DataFrame(pv).T for pk, pv in p_sample_dic.items()]
Weighted_d = []
for pindex, pr in enumerate(runsdf):
    for Ea, val in Extra.iteritems():
        if (k+1)*2**(pindex+2) == Ea:
            Weighted = []
            for Eb, val in Extra.iterrows():
                for pe, pval in pr.iterrows():
                    if pe == Eb:
                        if len(pr.T) < int(Extra[Ea][Eb]):
                            Weighted.append(np.mean(pr.T))
                        else:
                            Weighted.append((np.mean(pr.T[Eb][:int(Extra[Ea][Eb])])))
                    Weighted_dic = create_dict(CheckName, Weighted)
            Weighted_d.append(Weighted_dic)
    Weighted_d_dict=create_dict(p_sample_name,Weighted_d)
Weighted_d_dict[28672]['CheckMAEA26']=Weighted_d_dict[14336]['CheckMAEA26']
Weighted_d_dict[28672]['CheckMAEA25']=Weighted_d_dict[14336]['CheckMAEA25']

In [ ]:
Weighted_l = []
for wk,wv in Weighted_d_dict.items():
    CheckMAEs = []
    CheckMAENames = []
    for ae, av in AE_dic.items():
        for Jk, Jv in wv.items():
            if ae[-5:-3]==Jk[-3:-1] and ae[-1]==Jk[-1]:
                CheckMAEs.append(abs(Jv-av))
                CheckMAENames.append('CheckMAE'+ str(ae[2:4]) + str(ae[-1]))
    CheckMAEs_dic = create_dict(CheckMAENames, CheckMAEs)
    
    CheckMAE = []
    CheckMAE_name = []
    for f in functions:
        validkeys4 = []
        for Jmk, Jmv in CheckMAEs_dic.items():
            if Jmk[-3:-1]==f.__name__:
                validkeys4.append(Jmk)
        z2 = dict(filter(lambda i4:i4[0] in validkeys4, CheckMAEs_dic.items()))
        CheckMAE.append(sum(z2.values())/len(z2))
        CheckMAE_name.append('CheckMAE'+f.__name__)
    CheckMAE_dic = create_dict(CheckMAE_name, CheckMAE)
    Weighted_l.append(CheckMAE_dic)
WF_dic = create_dict(p_sample_name,Weighted_l)
WF_df = pd.DataFrame.from_dict(WF_dic)

In [ ]:
p_sample_Jan = []
qamples = []
for s in range (2,p):
    qamples.append(df.iloc[run*(-4+2**s):run*(-4+2**(s+1))].reset_index(drop=True))
    CheckMAE_mean = []
    
    run_samples = []
    for r in range (run):
        run_samples.append(qamples[s-2].iloc[int(r*(len(qamples[s-2])/run)):int((r+1)*(len(qamples[s-2].\
        index)/run))].reset_index(drop=True))
        
        sample_Matrices = []
        for m in range (n):
            sample_Matrices.append(run_samples[r].iloc[int(m*(len(run_samples[r].index)/n)):int((m+1)*\
            (len(run_samples[r].index)/n))].reset_index(drop=True))
                
        sample_Matrices_dic = create_dict(letters, sample_Matrices)
        
            
        mixed_Matrices = []
        mm_names = []
        for sm in range (0,len(sample_Matrices)):
            for sm1 in range (0,len(sample_Matrices)):
                if sm == sm1:
                    continue
                else:
                    for c in sample_Matrices[sm]:
                        mixed_Matrices.append(sample_Matrices[sm].copy())
                        mixed_Matrices[len(mixed_Matrices)-1][c]=sample_Matrices[sm1].copy()[c]
                        mm_names.append(str(letters[sm] + letters[sm1] + str(c+1)))
        mixed_Matrices_dic = create_dict(mm_names, mixed_Matrices)
        
        matrices_dic = {**sample_Matrices_dic, **mixed_Matrices_dic}
        
        names = []
        values = []
        names1 = []
        values1 = []
        names2 = []
        values2 = []
        for f in functions:
            for ss, zs in sample_Matrices_dic.items():
                names.append(f.__name__+str(ss))
                values.append(f(zs))
    
            for sq, zq in mixed_Matrices_dic.items():
                names1.append(f.__name__+str(sq))
                values1.append(f(zq))
            
            for sM, zM in matrices_dic.items():
                names2.append(f.__name__+str(sM))
                values2.append(f(zM))
                
        f_SM_dic = create_dict(names, values)
        f_MM_dic = create_dict(names1, values1)
        f_matrices_dic = create_dict(names2, values2)

        Check=[]
        CheckName = []
        for f in functions:
            for j in range(1,k+1):
                difference = []
                for mk, mz in f_matrices_dic.items():
                    if mk[0:2]==f.__name__:
                        validkeys3 = []
                        for fk1 in f_MM_dic.keys():
                            if len(mk)==3: 
                                if mk[2] == 'a' and fk1[0:3]==mk[0:3] and fk1[-1]==str(j):
                                    validkeys3.append(fk1)
                            else: 
                                if fk1[0:3]==mk[0:3] and fk1[-1]==mk[-1] and fk1!=mk and fk1>=mk and fk1[-1]==str(j):
                                    validkeys3.append(fk1)
                        z1 = dict(filter(lambda i:i[0] in validkeys3, f_MM_dic.items()))
                        for zk, zv in z1.items():
                            difference.append(0.5*(((mz-zv)**2).mean())/mz.var())
                Check.append(sum(difference)/len(difference))
                CheckName.append('Jansen'+ str(f.__name__) +'ST'+str(j))
        Check_dic = create_dict(CheckName, Check)
                    
        CheckMAEs = []
        CheckMAENames = []
        for ae, av in AE_dic.items():
            for Jk, Jv in Check_dic.items():
                if ae[-5:]==Jk[-5:]:
                    CheckMAEs.append(abs(Jv-av))
                    CheckMAENames.append('CheckMAE'+ str(ae[2:4]) + str(ae[-1]))
        CheckMAEs_dic = create_dict(CheckMAENames, CheckMAEs)
    
        CheckMAE = []
        CheckMAE_name = []
        for f in functions:
            validkeys4 = []
            for Jmk, Jmv in CheckMAEs_dic.items():
                if Jmk[-3:-1]==f.__name__:
                    validkeys4.append(Jmk)
            z2 = dict(filter(lambda i4:i4[0] in validkeys4, CheckMAEs_dic.items()))
            CheckMAE.append(sum(z2.values())/len(z2))
            CheckMAE_name.append('CheckMAE'+f.__name__)
        CheckMAE_dic = create_dict(CheckMAE_name, CheckMAE)
        CheckMAE_mean.append(CheckMAE_dic)
        
    CheckMAE_mean_dic = {Jmk1:[CheckMAE_mean[Jmv1][Jmk1] for Jmv1 in range(len(CheckMAE_mean))] for Jmk1 in CheckMAE_mean[0].keys()}
    for Jmk1,Jmv1 in CheckMAE_mean_dic.items():
        CheckMAE_mean_dic[Jmk1] = sum(CheckMAE_mean_dic[Jmk1])/len(CheckMAE_mean_dic[Jmk1])
        
    p_sample_Jan.append(CheckMAE_mean_dic)
p_sample_Jan_dic = create_dict(p_sample_name, p_sample_Jan)
Jansen_asym = pd.DataFrame.from_dict(p_sample_Jan_dic)

In [ ]:
writer = pd.ExcelWriter('C:/Users/samuele/Dropbox (Personal)/Research/Sensitivity Analysis/Jansen_fast.xlsx')
WF_df.to_excel(writer,'Jansen_fast')
Jansen_asym.to_excel(writer,'Jansen_asym')
writer.save()

In [55]:
for ind, row in WF_df.iterrows():
    for indR, row in Jansen_asym.iterrows():
        if ind[-5:] == indR[-5:]:
            x_vals = WF_df.columns.values
            x2_vals = Jansen_asym.columns.values
            y1 = WF_df.loc[ind]
            y2 = Jansen_asym.loc[indR]
            for i6 in range(0, len(x_vals), 1):
                plt.loglog(x_vals[i6:i6+2], y1[i6:i6+2], c = "b", marker = "o", label = 'this paper' if i6 == 0 else '')
                plt.loglog(x2_vals[i6:i6+2], y2[i6:i6+2], c = "r", marker = "o", label = 'Jan' if i6 == 0 else '')
                plt.title('ThispaperVsJansen_asym' + '_ST_' +str(ind[-2:]))
                plt.xlim(50,58000)
                plt.ylim(0,1)
            plt.legend()
            plt.savefig('ThispaperVsJansen_asym' + '_ST_' +str(ind[-2:])+".png", format="PNG")
            plt.close()

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\axes\_base.py:3449: UserWarning: Attempted to set non-positive ylimits for log-scale axis; invalid limits will be ignored.
  'Attempted to set non-positive ylimits for log-scale axis; '


In [ ]:
# Attempting at coding according to the difference with the Analytical Value rather than the relative ST
p_sample = []
p_sample_name = []

qamples = [df.iloc[srun[s-2]:(srun[s-2]+srun[s-1])].reset_index(drop=True) for s in range(2,p)]

run_samples = []
run_c_samples = []
for s in range (2, p-1):
    run_samples.append(qamples[s-2].iloc[:2**s*int(run*((p-s-1))/(p-3))].reset_index(drop=True))
    
    Check_mean = []
    run_samplesr = []
    for r in range (int(run*((p-s-1))/(p-3))):
        run_samplesr.append(run_samples[s-2].iloc[r*int(len(run_samples[s-2])/int(run*(p-s-1)/(p-3))):(r+1)*\
        int(len(run_samples[s-2])/int(run*(p-s-1)/(p-3)))].reset_index(drop=True))
        
        sample_Matrices = []
        for m in range (n):
            sample_Matrices.append(run_samplesr[r].iloc[int(m*(len(run_samplesr[r])/n)):int((m+1)*\
            (len(run_samplesr[r])/n))].reset_index(drop=True))
                
        sample_Matrices_dic = create_dict(letters, sample_Matrices)
        
        mixed_Matrices = []
        mm_names = []
        for sm in range (0,len(sample_Matrices)):
            for sm1 in range (0,len(sample_Matrices)):
                if sm == sm1:
                    continue
                else:
                    for c in sample_Matrices[sm]:
                        mixed_Matrices.append(sample_Matrices[sm].copy())
                        mixed_Matrices[len(mixed_Matrices)-1][c]=sample_Matrices[sm1].copy()[c]
                        mm_names.append(str(letters[sm] + letters[sm1] + str(c+1)))
        mixed_Matrices_dic = create_dict(mm_names, mixed_Matrices)
        
        matrices_dic = {**sample_Matrices_dic, **mixed_Matrices_dic}
        
        names = []
        names1 = []
        names2 = []
        values = []
        values1 = []
        values2 = []
        for f in functions:
            for ss, zs in sample_Matrices_dic.items():
                names.append(f.__name__+str(ss))
                values.append(f(zs))

            for sq, zq in mixed_Matrices_dic.items():
                names1.append(f.__name__+str(sq))
                values1.append(f(zq))
            
            for sM, zM in matrices_dic.items():
                names2.append(f.__name__+str(sM))
                values2.append(f(zM))
                
        f_SM_dic = create_dict(names, values)
        f_MM_dic = create_dict(names1, values1)
        f_matrices_dic = create_dict(names2, values2)

        Check=[]
        CheckName = []
        for f in functions:
            for j in range(1,k+1):
                difference = []
                for mk, mz in f_matrices_dic.items():
                    if mk[0:2]==f.__name__:
                        validkeys3 = []
                        for fk1 in f_MM_dic.keys():
                            if len(mk)==3: 
                                if mk[2] == 'a' and fk1[0:3]==mk[0:3] and fk1[-1]==str(j):
                                    validkeys3.append(fk1)
                            else: 
                                if fk1[0:3]==mk[0:3] and fk1[-1]==mk[-1] and fk1!=mk and fk1>=mk and fk1[-1]==str(j):
                                    validkeys3.append(fk1)
                        z1 = dict(filter(lambda i:i[0] in validkeys3, f_MM_dic.items()))
                        for zk, zv in z1.items():
                            difference.append(0.5*(((mz-zv)**2).mean())/mz.var())
                Check.append(sum(difference)/len(difference))
                CheckName.append('CheckMAE'+ str(f.__name__) + str(j))
        Check_dic = create_dict(CheckName, Check)
        Check_mean.append(Check_dic)
    
        CheckMAEs = []
        CheckMAENames = []
        for ae, av in AE_dic.items():
            for Jk, Jv in Check_dic.items():
                if ae[-5:-3]==Jk[-3:-1] and ae[-1]==Jk[-1]:
                    CheckMAEs.append(abs(Jv-av))
                    CheckMAENames.append('CheckMAE'+ str(ae[2:4]) + str(ae[-1]))
        CheckMAEs_dic = create_dict(CheckMAENames, CheckMAEs)
        
        CheckMAE_mean = []
        CheckMAE_mean.append(CheckMAEs_dic)
        
    CheckMAE_mean_dic = {Jmk1:[CheckMAE_mean[Jmv1][Jmk1] for Jmv1 in range(len(CheckMAE_mean))] for Jmk1 in CheckMAE_mean[0].keys()}
    for Jmk1,Jmv1 in CheckMAE_mean_dic.items():
        CheckMAE_mean_dic[Jmk1] = sum(CheckMAE_mean_dic[Jmk1])/len(CheckMAE_mean_dic[Jmk1])
    
    p_sample.append(CheckMAE_mean_dic)
    p_sample_name.append((k+1)*2**s)
p_sample_dic = create_dict(p_sample_name, p_sample)

In [ ]:
runsdf = [pd.DataFrame(pv).T for pk, pv in p_sample_dic.items()]
Weighted_d = []
for pindex, pr in enumerate(runsdf):
    for Ea, val in Extra.iteritems():
        if (k+1)*2**(pindex+2) == Ea:
            Weighted = []
            for Eb, val in Extra.iterrows():
                for pe, pval in pr.iterrows():
                    if pe == Eb:
                        if len(pr.T) < int(Extra[Ea][Eb]):
                            Weighted.append(np.mean(pr.T))
                        else:
                            Weighted.append((np.mean(pr.T[Eb][:int(Extra[Ea][Eb])])))
                    Weighted_dic = create_dict(CheckName, Weighted)
            Weighted_d.append(Weighted_dic)
    Weighted_d_dict=create_dict(p_sample_name,Weighted_d)
Weighted_d_dict[57344]['CheckMAEA16']=Weighted_d_dict[28672]['CheckMAEA16']
Weighted_d_dict[57344]['CheckMAEA24']=Weighted_d_dict[28672]['CheckMAEA24']
Weighted_d_dict[57344]['CheckMAEA25']=Weighted_d_dict[28672]['CheckMAEA25']
Weighted_d_dict[57344]['CheckMAEA26']=Weighted_d_dict[28672]['CheckMAEA26']

In [ ]:
Weighted_l = []
for wk,wv in Weighted_d_dict.items():
    CheckMAEs = []
    CheckMAENames = []
    for ae, av in AE_dic.items():
        for Jk, Jv in wv.items():
            if ae[-5:-3]==Jk[-3:-1] and ae[-1]==Jk[-1]:
                CheckMAEs.append(abs(Jv-av))
                CheckMAENames.append('CheckMAE'+ str(ae[2:4]) + str(ae[-1]))
    CheckMAEs_dic = create_dict(CheckMAENames, CheckMAEs)
    
    CheckMAE = []
    CheckMAE_name = []
    for f in functions:
        validkeys4 = []
        for Jmk, Jmv in CheckMAEs_dic.items():
            if Jmk[-3:-1]==f.__name__:
                validkeys4.append(Jmk)
        z2 = dict(filter(lambda i4:i4[0] in validkeys4, CheckMAEs_dic.items()))
        CheckMAE.append(sum(z2.values())/len(z2))
        CheckMAE_name.append('CheckMAE'+f.__name__)
    CheckMAE_dic = create_dict(CheckMAE_name, CheckMAE)
    Weighted_l.append(CheckMAE_dic)
WF2_dic = create_dict(p_sample_name,Weighted_l)
WF2_df = pd.DataFrame.from_dict(WF2_dic)

In [ ]:
writer = pd.ExcelWriter('C:/Users/samuele/Dropbox (Personal)/Research/Sensitivity Analysis/Jansen_fast.xlsx')
WF_df.to_excel(writer,'Jansen_fast')
WF2_df.to_excel(writer,'Jansen_fast_MAE')
writer.save()

In [ ]:
for ind, row in WF_df.iterrows():
    for indR, row in WF2_df.iterrows():
        if ind[-5:] == indR[-5:]:
            x_vals = WF_df.columns.values
            x2_vals = WF2_df.columns.values
            y1 = WF_df.loc[ind]
            y2 = WF2_df.loc[indR]
            for i6 in range(0, len(x_vals), 1):
                plt.loglog(x_vals[i6:i6+2], y1[i6:i6+2], c = "b", marker = "o", label = 'this paper' if i6 == 0 else '')
                plt.loglog(x2_vals[i6:i6+2], y2[i6:i6+2], c = "r", marker = "o", label = 'this paper_MAE' if i6 == 0 else '')
                plt.title('STvsMAE' + '_ST_' +str(ind[-2:]))
                plt.xlim(110,58000)
                plt.ylim(0,1)
            plt.legend()
            plt.savefig('ThispaperVsMAE' + '_ST_' +str(ind[-2:])+".png", format="PNG")
            plt.close()